In [ ]:
%matplotlib inline
from pak.datasets.Market1501 import Market1501
import matplotlib.pyplot as plt
from keras.models import load_model
import sys
sys.path.insert(0, '../')
from reid import reid

root = 'data_storage'
m1501 = Market1501(root)
X, Y = m1501.get_train()



model = reid.ReId()


def predict(A, B):
    pass


# ==== same ====
im1 = X[0]
im2 = X[200]

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(121); ax.axis('off')
ax.imshow(im1)
ax = fig.add_subplot(122); ax.axis('off')
ax.imshow(im2)
plt.show()

In [ ]:
score = model.predict(im1, im2)